In [1]:
#import the necessities
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
#call the data from the tab delimited txt file
mtg = pd.read_csv('D:/Downloads/cardsutf8.txt', sep='\t', header=0, low_memory=False)

In [32]:
white_sample = pd.DataFrame()
blue_sample = pd.DataFrame()
black_sample = pd.DataFrame()
red_sample = pd.DataFrame()
green_sample = pd.DataFrame()

white_sample_2 = pd.DataFrame()
blue_sample_2 = pd.DataFrame()
black_sample_2 = pd.DataFrame()
red_sample_2 = pd.DataFrame()
green_sample_2 = pd.DataFrame()

In [50]:
#confirm the data
print(mtg.shape)
print(mtg.columns)
print(mtg.head())

(55943, 75)
Index(['index', 'id', 'artist', 'asciiName', 'availability', 'borderColor',
       'cardKingdomFoilId', 'cardKingdomId', 'colorIdentity', 'colorIndicator',
       'colors', 'convertedManaCost', 'duelDeck', 'edhrecRank',
       'faceConvertedManaCost', 'faceName', 'flavorName', 'flavorText',
       'frameEffects', 'frameVersion', 'hand', 'hasAlternativeDeckLimit',
       'hasContentWarning', 'hasFoil', 'hasNonFoil', 'isAlternative',
       'isFullArt', 'isOnlineOnly', 'isOversized', 'isPromo', 'isReprint',
       'isReserved', 'isStarter', 'isStorySpotlight', 'isTextless',
       'isTimeshifted', 'keywords', 'layout', 'leadershipSkills', 'life',
       'loyalty', 'manaCost', 'mcmId', 'mcmMetaId', 'mtgArenaId',
       'mtgjsonV4Id', 'mtgoFoilId', 'mtgoId', 'multiverseId', 'name', 'number',
       'originalReleaseDate', 'originalText', 'originalType', 'otherFaceIds',
       'power', 'printings', 'promoTypes', 'purchaseUrls', 'rarity',
       'scryfallId', 'scryfallIllustration

In [3]:
#we're only interested in a few items, so we'll drop the
#unnecessary values and columns, checking the shape to
#confirm values were dropped
filtered_mtg = mtg[mtg['borderColor'] == 'black']
print(filtered_mtg.shape)

(48870, 75)


In [4]:
#we're only interested in creatures so we'll
#drop any values that aren't creatures
filtered_mtg_1 = filtered_mtg[filtered_mtg['types'].str.contains('Creature')]
filtered_mtg_1.shape

(22997, 75)

In [5]:
#let's drop the duplicates
#for this experiment
filtered_mtg_2 = filtered_mtg_1.drop_duplicates(['name'])
filtered_mtg_2.shape

(11090, 75)

In [6]:
#some of the values for power are not
#ints. because they are variable
#we'll drop the values
filtered_mtg_3 = filtered_mtg_2[(filtered_mtg_2['power'] != '*') & (filtered_mtg_2['power'] != '1+*') &
                               (filtered_mtg_2['power'] != '2+*') & (filtered_mtg_2['power'] != '?')]
filtered_mtg_3['power'] = filtered_mtg_3['power'].astype(int)
filtered_mtg_3.shape

(10941, 75)

In [9]:
#let's get only the columns of interest
dataset = filtered_mtg_3[['borderColor', 'colors', 'convertedManaCost', 'faceConvertedManaCost',
                          'faceName',  'manaCost', 'name', 'power','side', 'subtypes', 'supertypes', 'text', 
                          'toughness', 'type', 'types']]
dataset.head()

,borderColor,colors,convertedManaCost,faceConvertedManaCost,faceName,manaCost,name,power,side,subtypes,supertypes,text,toughness,type,types
1,black,U,3.0,NaN,NaN,{1}{U}{U},Academy Researchers,2,NaN,"Human,Wizard",NaN,When Academy Researchers enters the battlefiel...,2,Creature â€” Human Wizard,Creature
6,black,U,5.0,NaN,NaN,{3}{U}{U},Air Elemental,4,NaN,Elemental,NaN,Flying,4,Creature â€” Elemental,Creature
8,black,U,3.0,NaN,NaN,{1}{U}{U},Ambassador Laquatus,1,NaN,"Merfolk,Wizard",Legendary,{3}: Target player mills three cards.,3,Legendary Creature â€” Merfolk Wizard,Creature
9,black,R,4.0,NaN,NaN,{3}{R},Anaba Bodyguard,2,NaN,Minotaur,NaN,First strike (This creature deals combat damag...,3,Creature â€” Minotaur,Creature
11,black,W,7.0,NaN,NaN,{5}{W}{W},Ancestor's Chosen,4,NaN,"Human,Cleric",NaN,First strike (This creature deals combat damag...,4,Creature â€” Human Cleric,Creature


In [56]:
#let's see all the unique values for colors
print(dataset['colors'].unique())

['U' 'R' 'W' 'B' 'G' nan 'G,U' 'B,G,U,W' 'B,U' 'B,R,U,W' 'R,U' 'B,R' 'U,W'
 'G,R' 'B,R,W' 'B,G,R' 'B,G' 'G,W' 'G,R,U' 'B,U,W' 'B,G,R,U,W' 'R,W' 'B,W'
 'B,R,U' 'G,U,W' 'G,R,W' 'B,G,U' 'R,U,W' 'B,G,W' 'B,G,R,U' 'G,R,U,W'
 'B,G,R,W' 'W,G']


In [57]:
#sort by colors
color_sort = dataset.sort_values(by='colors')
color_sort.head()

,id,borderColor,colors,convertedManaCost,faceConvertedManaCost,faceName,manaCost,name,power,side,subtypes,supertypes,text,toughness,type,types
51008,51009,black,B,5.0,NaN,NaN,{4}{B},Plague Dogs,3,NaN,"Zombie,Dog",NaN,"When Plague Dogs dies, all creatures get -1/-1...",3,Creature â€” Zombie Dog,Creature
7406,7407,black,B,3.0,NaN,NaN,{2}{B},Malakir Familiar,2,NaN,Bat,NaN,"Flying, deathtouch\nWhenever you gain life, Ma...",1,Creature â€” Bat,Creature
23574,23575,black,B,4.0,NaN,NaN,{3}{B},Trench Wurm,3,NaN,Wurm,NaN,"{2}{R}, {T}: Destroy target nonbasic land.",3,Creature â€” Wurm,Creature
23580,23581,black,B,4.0,NaN,NaN,{2}{B}{B},Tsabo's Assassin,1,NaN,"Zombie,Assassin",NaN,{T}: Destroy target creature if it shares a co...,1,Creature â€” Zombie Assassin,Creature
50631,50632,black,B,3.0,NaN,NaN,{2}{B},Basal Sliver,2,NaN,Sliver,NaN,"All Slivers have ""Sacrifice this permanent: Ad...",2,Creature â€” Sliver,Creature


In [58]:
#reset the index
colors_index = color_sort.reset_index()
colors_index = colors_index.drop(['index'], axis=1)
colors_index.head()

,id,borderColor,colors,convertedManaCost,faceConvertedManaCost,faceName,manaCost,name,power,side,subtypes,supertypes,text,toughness,type,types
0,51009,black,B,5.0,NaN,NaN,{4}{B},Plague Dogs,3,NaN,"Zombie,Dog",NaN,"When Plague Dogs dies, all creatures get -1/-1...",3,Creature â€” Zombie Dog,Creature
1,7407,black,B,3.0,NaN,NaN,{2}{B},Malakir Familiar,2,NaN,Bat,NaN,"Flying, deathtouch\nWhenever you gain life, Ma...",1,Creature â€” Bat,Creature
2,23575,black,B,4.0,NaN,NaN,{3}{B},Trench Wurm,3,NaN,Wurm,NaN,"{2}{R}, {T}: Destroy target nonbasic land.",3,Creature â€” Wurm,Creature
3,23581,black,B,4.0,NaN,NaN,{2}{B}{B},Tsabo's Assassin,1,NaN,"Zombie,Assassin",NaN,{T}: Destroy target creature if it shares a co...,1,Creature â€” Zombie Assassin,Creature
4,50632,black,B,3.0,NaN,NaN,{2}{B},Basal Sliver,2,NaN,Sliver,NaN,"All Slivers have ""Sacrifice this permanent: Ad...",2,Creature â€” Sliver,Creature


In [10]:
#for this experiemt, we will focus only on
#the cards with a single color
#in future analyses, multi-color
#cards can also be analyzed
white_creatures = dataset[dataset['colors'] == 'W']
white_creatures.insert(0, 'unique_id', range(1, 1 + len(white_creatures)))
display(white_creatures.head())
display(white_creatures.tail())
print(white_creatures.shape)

,unique_id,borderColor,colors,convertedManaCost,faceConvertedManaCost,faceName,manaCost,name,power,side,subtypes,supertypes,text,toughness,type,types
11,1,black,W,7.0,NaN,NaN,{5}{W}{W},Ancestor's Chosen,4,NaN,"Human,Cleric",NaN,First strike (This creature deals combat damag...,4,Creature â€” Human Cleric,Creature
13,2,black,W,5.0,NaN,NaN,{4}{W},Angel of Mercy,3,NaN,Angel,NaN,Flying\nWhen Angel of Mercy enters the battlef...,3,Creature â€” Angel,Creature
19,3,black,W,2.0,NaN,NaN,{1}{W},Angelic Wall,0,NaN,Wall,NaN,Defender (This creature can't attack.)\nFlying,4,Creature â€” Wall,Creature
31,4,black,W,4.0,NaN,NaN,{3}{W},Aven Cloudchaser,2,NaN,"Bird,Soldier",NaN,Flying (This creature can't be blocked except ...,2,Creature â€” Bird Soldier,Creature
37,5,black,W,4.0,NaN,NaN,{3}{W},Ballista Squad,2,NaN,"Human,Rebel",NaN,"{X}{W}, {T}: Ballista Squad deals X damage to ...",2,Creature â€” Human Rebel,Creature


,unique_id,borderColor,colors,convertedManaCost,faceConvertedManaCost,faceName,manaCost,name,power,side,subtypes,supertypes,text,toughness,type,types
55777,1822,black,W,4.0,NaN,NaN,{3}{W},Prowling Felidar,2,NaN,"Cat,Beast",NaN,Vigilance\nLandfall â€” Whenever a land enters...,3,Creature â€” Cat Beast,Creature
55813,1823,black,W,1.0,NaN,NaN,{W},Sea Gate Banneret,1,NaN,"Kor,Warrior",NaN,{4}{W}: Creatures you control get +1/+1 until ...,2,Creature â€” Kor Warrior,Creature
55835,1824,black,W,5.0,NaN,NaN,{4}{W},Shepherd of Heroes,3,NaN,"Angel,Cleric",NaN,Flying\nWhen Shepherd of Heroes enters the bat...,4,Creature â€” Angel Cleric,Creature
55842,1825,black,W,2.0,NaN,Skyclave Cleric,{1}{W},Skyclave Cleric // Skyclave Basilica,1,a,"Kor,Cleric",NaN,"When Skyclave Cleric enters the battlefield, y...",3,Creature â€” Kor Cleric,Creature
55892,1826,black,W,3.0,NaN,NaN,{2}{W},Tazeem Raptor,2,NaN,Bird,NaN,Flying\nWhen Tazeem Raptor enters the battlefi...,2,Creature â€” Bird,Creature


(1826, 16)


In [22]:
#we will continue this for each color
#second is blue
blue_creatures = dataset[dataset['colors'] == 'U']
blue_creatures.insert(0, 'unique_id', range(1, 1 + len(blue_creatures)))
display(blue_creatures.head())
print(blue_creatures.shape)

,unique_id,borderColor,colors,convertedManaCost,faceConvertedManaCost,faceName,manaCost,name,power,side,subtypes,supertypes,text,toughness,type,types
1,1,black,U,3.0,NaN,NaN,{1}{U}{U},Academy Researchers,2,NaN,"Human,Wizard",NaN,When Academy Researchers enters the battlefiel...,2,Creature â€” Human Wizard,Creature
6,2,black,U,5.0,NaN,NaN,{3}{U}{U},Air Elemental,4,NaN,Elemental,NaN,Flying,4,Creature â€” Elemental,Creature
8,3,black,U,3.0,NaN,NaN,{1}{U}{U},Ambassador Laquatus,1,NaN,"Merfolk,Wizard",Legendary,{3}: Target player mills three cards.,3,Legendary Creature â€” Merfolk Wizard,Creature
23,4,black,U,6.0,NaN,NaN,{3}{U}{U}{U},Arcanis the Omnipotent,3,NaN,Wizard,Legendary,{T}: Draw three cards.\n{2}{U}{U}: Return Arca...,4,Legendary Creature â€” Wizard,Creature
33,5,black,U,4.0,NaN,NaN,{3}{U},Aven Fisher,2,NaN,"Bird,Soldier",NaN,Flying (This creature can't be blocked except ...,2,Creature â€” Bird Soldier,Creature


(1594, 16)


In [61]:
#next is black
black_creatures = dataset[dataset['colors'] == 'B']
black_creatures = black_creatures.reset_index()
black_creatures = black_creatures.drop(['index'], axis=1)
print(black_creatures.head())
print(black_creatures.shape)

    id borderColor colors  convertedManaCost  faceConvertedManaCost faceName  \
0   25       black      B                6.0                    NaN      NaN   
1   53       black      B                4.0                    NaN      NaN   
2  103       black      B                3.0                    NaN      NaN   
3  109       black      B                4.0                    NaN      NaN   
4  110       black      B                2.0                    NaN      NaN   

    manaCost                name  power side                subtypes  \
0  {4}{B}{B}   Ascendant Evincar      3  NaN           Vampire,Noble   
1     {3}{B}          Bog Wraith      3  NaN                  Wraith   
2     {2}{B}  Doomed Necromancer      2  NaN  Human,Cleric,Mercenary   
3     {3}{B}     Dross Crocodile      5  NaN        Zombie,Crocodile   
4     {1}{B}    Drudge Skeletons      1  NaN                Skeleton   

  supertypes                                               text toughness  \
0  Legend

In [62]:
#then red
red_creatures = dataset[dataset['colors'] == 'R']
red_creatures = red_creatures.reset_index()
red_creatures = red_creatures.drop(['index'], axis=1)
print(red_creatures.head())
print(red_creatures.shape)

    id borderColor colors  convertedManaCost  faceConvertedManaCost faceName  \
0   10       black      R                4.0                    NaN      NaN   
1   51       black      R                8.0                    NaN      NaN   
2   52       black      R                3.0                    NaN      NaN   
3   55       black      R                3.0                    NaN      NaN   
4  112       black      R                1.0                    NaN      NaN   

    manaCost                name  power side          subtypes supertypes  \
0     {3}{R}     Anaba Bodyguard      2  NaN          Minotaur        NaN   
1  {6}{R}{R}  Bloodfire Colossus      6  NaN             Giant        NaN   
2     {2}{R}   Bloodrock Cyclops      3  NaN           Cyclops        NaN   
3     {2}{R}  Bogardan Firefiend      2  NaN  Elemental,Spirit        NaN   
4        {R}        Duct Crawler      1  NaN            Insect        NaN   

                                                text tou

In [63]:
#finishing the colors is green
green_creatures = dataset[dataset['colors'] == 'G']
green_creatures = green_creatures.reset_index()
green_creatures = green_creatures.drop(['index'], axis=1)
print(green_creatures.head())
print(green_creatures.shape)

   id borderColor colors  convertedManaCost  faceConvertedManaCost faceName  \
0  30       black      G                8.0                    NaN      NaN   
1  46       black      G                1.0                    NaN      NaN   
2  60       black      G                2.0                    NaN      NaN   
3  68       black      G                3.0                    NaN      NaN   
4  87       black      G                6.0                    NaN      NaN   

    manaCost               name  power side           subtypes supertypes  \
0  {6}{G}{G}    Avatar of Might      8  NaN             Avatar        NaN   
1        {G}  Birds of Paradise      0  NaN               Bird        NaN   
2     {1}{G}      Canopy Spider      1  NaN             Spider        NaN   
3     {2}{G}    Civic Wayfinder      2  NaN  Elf,Druid,Warrior        NaN   
4  {4}{G}{G}          Craw Wurm      6  NaN               Wurm        NaN   

                                                text toughness

In [34]:
#for this experiemt, we will focus only on
#the cards with a single color
#in future analyses, multi-color
#cards can also be analyzed
white_creatures = dataset[dataset['colors'] == 'W']
white_creatures = white_creatures.reset_index()
white_creatures = white_creatures.drop(['index'], axis=1)
#we will continue this for each color
#second is blue
blue_creatures = dataset[dataset['colors'] == 'U']
blue_creatures = blue_creatures.reset_index()
blue_creatures = blue_creatures.drop(['index'], axis=1)
#next black
black_creatures = dataset[dataset['colors'] == 'B']
black_creatures = black_creatures.reset_index()
black_creatures = black_creatures.drop(['index'], axis=1)
#then red
red_creatures = dataset[dataset['colors'] == 'R']
red_creatures = red_creatures.reset_index()
red_creatures = red_creatures.drop(['index'], axis=1)
#finishing the colors is green
green_creatures = dataset[dataset['colors'] == 'G']
green_creatures = green_creatures.reset_index()
green_creatures = green_creatures.drop(['index'], axis=1)
#creating a random sampling from each color

In [82]:
white_creatures.head()

In [120]:
white_sample_2 = pd.DataFrame()
blue_sample_2 = pd.DataFrame()
black_sample_2 = pd.DataFrame()
red_sample_2 = pd.DataFrame()
green_sample_2 = pd.DataFrame()

In [121]:
white_sample = pd.DataFrame()
blue_sample = pd.DataFrame()
black_sample = pd.DataFrame()
red_sample = pd.DataFrame()
green_sample = pd.DataFrame()

random_set = []
for item in range(400):
    #create random number here
    num = random.randint(1, white_creatures.shape[0])
    #check to see if that number has been selected before or not
    #if it has been selected, pick another random number
    while num in random_set:
        num = random.randint(1, white_creatures.shape[0])
    #if it hasn't, add it to the list and add the corresponding
    #creature information to the statistical sampling set
    random_set.append(num)
    white_sample = white_sample.append(white_creatures['unique_id'] == num])
#repeating the process below
#I would love to put these into for loops
#but am unable to use multiple variables
#with dataframes at present
random_set = []
for item in range(400):
    num = random.randint(1, blue_creatures.shape[0])
    while num in random_set:
        num = random.randint(1, blue_creatures.shape[0])
    random_set.append(num)
    blue_sample = blue_sample.append(blue_creatures.['unique_id'] == num])
random_set = []
for item in range(400):
    num = random.randint(1, black_creatures.shape[0])
    while num in random_set:
        num = random.randint(1, black_creatures.shape[0])
    random_set.append(num)
    black_sample = black_sample.append(black_creatures.loc[[num]])
random_set = []
for item in range(400):
    #create random number here
    num = random.randint(0, red_creatures.shape[0])
    #check to see if that number has been selected before or not
    while num in random_set:
        num = random.randint(0, red_creatures.shape[0])
    random_set.append(num)
    red_sample = red_sample.append(red_creatures.loc[[num]])
random_set = []
for item in range(400):
    #create random number here
    num = random.randint(0, green_creatures.shape[0])
    #check to see if that number has been selected before or not
    while num in random_set:
        num = random.randint(0, green_creatures.shape[0])
    random_set.append(num)
    green_sample = green_sample.append(green_creatures.loc[[num]])
    #running the test again to allow for A/A analysis
#creating a random sampling from each color
random_set = []
for item in range(400):
    num = random.randint(0, white_creatures.shape[0])
    while num in random_set:
        num = random.randint(0, white_creatures.shape[0])
    random_set.append(num)
    white_sample_2 = white_sample_2.append(white_creatures.loc[[num]])
#repeating the process below
#I would love to put these into for loops
#but am unable to use multiple variables
#with dataframes at present
random_set = []
for item in range(400):
    num = random.randint(0, blue_creatures.shape[0])
    while num in random_set:
        num = random.randint(0, blue_creatures.shape[0])
    random_set.append(num)
    blue_sample_2 = blue_sample_2.append(blue_creatures.loc[[num]])
random_set = []
for item in range(400):
    num = random.randint(0, black_creatures.shape[0])
    while num in random_set:
        num = random.randint(0, black_creatures.shape[0])
    random_set.append(num)
    black_sample_2 = black_sample_2.append(black_creatures.loc[[num]])
random_set = []
for item in range(400):
    #create random number here
    num = random.randint(0, red_creatures.shape[0])
    #check to see if that number has been selected before or not
    while num in random_set:
        num = random.randint(0, red_creatures.shape[0])
    random_set.append(num)
    red_sample_2 = red_sample_2.append(red_creatures.loc[[num]])
random_set = []
for item in range(400):
    #create random number here
    num = random.randint(0, green_creatures.shape[0])
    #check to see if that number has been selected before or not
    while num in random_set:
        num = random.randint(0, green_creatures.shape[0])
    random_set.append(num)
    green_sample_2 = green_sample_2.append(green_creatures.loc[[num]])

KeyError: "None of [Int64Index([1594], dtype='int64')] are in the [index]"

In [29]:
## create datasets for each individual color

#for this experiemt, we will focus only on
#the cards with a single color
#in future analyses, multi-color
#cards can also be analyzed
white_creatures = dataset[dataset['colors'] == 'W']
white_creatures.insert(0, 'unique_id', range(1, 1 + len(white_creatures)))
#we will continue this for each color
#second is blue
blue_creatures = dataset[dataset['colors'] == 'U']
blue_creatures.insert(0, 'unique_id', range(1, 1 + len(blue_creatures)))
#next black
black_creatures = dataset[dataset['colors'] == 'B']
black_creatures.insert(0, 'unique_id', range(1, 1 + len(black_creatures)))
#then red
red_creatures = dataset[dataset['colors'] == 'R']
red_creatures.insert(0, 'unique_id', range(1, 1 + len(red_creatures)))
#finishing the colors is green
green_creatures = dataset[dataset['colors'] == 'G']
green_creatures.insert(0, 'unique_id', range(1, 1 + len(green_creatures)))

In [33]:
#creating a random sampling from each color
random_set = []
for item in range(400):
    #create random number here
    num = random.randint(1, white_creatures.shape[0])
    #check to see if that number has been selected before or not
    #if it has been selected, pick another random number
    while num in random_set:
        num = random.randint(1, white_creatures.shape[0])
    #if it hasn't, add it to the list and add the corresponding
    #creature information to the statistical sampling set
    random_set.append(num)
    white_sample = white_sample.append(white_creatures[white_creatures['unique_id'] == num])
#repeating the process below
#I would love to put these into for loops
#but am unable to use multiple variables
#with dataframes at present
random_set = []
for item in range(400):
    num = random.randint(1, blue_creatures.shape[0])
    while num in random_set:
        num = random.randint(1, blue_creatures.shape[0])
    random_set.append(num)
    blue_sample = blue_sample.append([blue_creatures['unique_id'] == num])
    random_set = []
for item in range(400):
    num = random.randint(1, black_creatures.shape[0])
    while num in random_set:
        num = random.randint(1, black_creatures.shape[0])
    random_set.append(num)
    black_sample = black_sample.append([black_creatures['unique_id'] == num])
random_set = []
for item in range(400):
    #create random number here
    num = random.randint(1, red_creatures.shape[0])
    #check to see if that number has been selected before or not
    while num in random_set:
        num = random.randint(1, red_creatures.shape[0])
    random_set.append(num)
    red_sample = red_sample.append([red_creatures['unique_id'] == num])
random_set = []
for item in range(400):
    #create random number here
    num = random.randint(1, green_creatures.shape[0])
    #check to see if that number has been selected before or not
    while num in random_set:
        num = random.randint(1, green_creatures.shape[0])
    random_set.append(num)
    green_sample = green_sample.append([green_creatures['unique_id'] == num])

In [ ]:
#running the test again to allow for A/A analysis
#creating a random sampling from each color
random_set = []
for item in range(400):
    num = random.randint(1, white_creatures.shape[0])
    while num in random_set:
        num = random.randint(1, white_creatures.shape[0])
    random_set.append(num)
    white_sample_2 = white_sample_2.append([white_creatures['unique_id'] == num])
random_set = []
for item in range(400):
    num = random.randint(1, blue_creatures.shape[0])
    while num in random_set:
        num = random.randint(1, blue_creatures.shape[0])
    random_set.append(num)
    blue_sample_2 = blue_sample_2.append([blue_creatures['unique_id'] == num])
random_set = []
for item in range(400):
    num = random.randint(1, black_creatures.shape[0])
    while num in random_set:
        num = random.randint(1, black_creatures.shape[0])
    random_set.append(num)
    black_sample_2 = black_sample_2.append([black_creatures['unique_id'] == num])
random_set = []
for item in range(400):
    #create random number here
    num = random.randint(1, red_creatures.shape[0])
    #check to see if that number has been selected before or not
    while num in random_set:
        num = random.randint(1, red_creatures.shape[0])
    random_set.append(num)
    red_sample_2 = red_sample_2.append([red_creatures['unique_id'] == num])
random_set = []
for item in range(400):
    #create random number here
    num = random.randint(1, green_creatures.shape[0])
    #check to see if that number has been selected before or not
    while num in random_set:
        num = random.randint(1, green_creatures.shape[0])
    random_set.append(num)
    green_sample_2 = green_sample_2.append([green_creatures['unique_id'] == num])

In [26]:
#getting the information from each power
print('The statistics for the first sampling of White creatures are')
print(white_sample['power'].describe())

The statistics for the first sampling of White creatures are
count    400.000000
mean       2.067500
std        1.046749
min        0.000000
25%        1.000000
50%        2.000000
75%        3.000000
max        6.000000
Name: power, dtype: float64


In [ ]:
#getting the information from each power
print('The statistics for the first sampling of White creatures are')
print(white_sample['power'].describe())
print('\n The statistics for the second sampling of White creatures are')
print(white_sample_2['power'].describe())

print('The statistics for the first sampling of Blue creatures are')
print(blue_sample['power'].describe())
print('\n The statistics for the second sampling of Blue creatures are')
print(blue_sample_2['power'].describe())

print('The statistics for the first sampling of Black creatures are')
print(black_sample['power'].describe())
print('\n The statistics for the second sampling of Black creatures are')
print(black_sample_2['power'].describe())

print('The statistics for the first sampling of Red creatures are')
print(red_sample['power'].describe())
print('\n The statistics for the second sampling of Red creatures are')
print(red_sample_2['power'].describe())

print('The statistics for the first sampling of Green creatures are')
print(green_sample['power'].describe())
print('\n The statistics for the second sampling of Green creatures are')
print(green_sample_2['power'].describe())

In [73]:
white_sample['id'].nunique()

400

In [ ]:
#create dfs and lists to allow for functions later